In [2]:
import pandas as pd
import numpy as np
import datetime
import os
import paramiko
import glob
print(datetime.datetime.now())
os.getcwd()


2019-12-18 16:25:05.125898


'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Experian_Audience'

In [4]:
list_seg_to_upload=glob.glob("./uploaded/*.csv")
list_seg_to_upload=[x for x in list_seg_to_upload if "SOTF" in x]
list_seg_to_upload=[x for x in list_seg_to_upload if "50klower" not in x.lower()]
print(len(list_seg_to_upload))
list_seg_to_upload

4


['./uploaded/ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseOwner.csv',
 './uploaded/ExperianSegment_2019Q4_SOTF_HHI100KHigher_HouseOwner.csv',
 './uploaded/ExperianSegment_2019Q4_SOTF_HHI100KHigher_HouseRents.csv',
 './uploaded/ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseRents.csv']

In [5]:
df=pd.read_csv(list_seg_to_upload[0],dtype=str,nrows=10)
df

,email_address_hash,segment
0,dd53075d06b1c045bb7b5756adf7e39d1ef26b24022b17...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...
1,1347f205646ac748109d5c73a20d43058a0d5a3980498a...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...
2,f5adcf28ff884986da48f6bc4f394191832ebed04ce9d7...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...
3,c33a06a75bd813daf1fb117c5c0ed7ce90470122287fdd...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...
4,d06c34399123c8200b7d0794c19f94f7a29fb3b4c4feef...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...
5,f87284450c0857338ecd61bbf4c4fc7493d1256d9c4b5b...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...
6,86cec84489122b38f7295e43979ec9e03b64f3af28b843...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...
7,768c740dd8d98001840fea229f9dacc206a43ceb402812...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...
8,4fb574e599dd00478fa32d7f32b898488b22c87a8a3e0e...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...
9,372a17c2aab2206f9e52fcee8d3c66b844d6bc097bbfc6...,ExperianSegment_2019Q4_SOTF_HHI50to100K_HouseO...


In [3]:
# LR
host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [8]:
for local_file in list_seg_to_upload:
    basename=os.path.basename(local_file)
    remote_path="/uploads/big_lots_onboarding/"+basename
    sftp.put(local_file,remote_path)
sftp.close()
transport.close()

In [9]:
# FB

import pandas as pd
import numpy as np
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import datetime
import os
import glob
BigLotsAccount_id="act_271491453638620"

jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token="EAAkEB3l7ONYBALOwTjZA6ZA69KGshjjZCghKd2FhqZCVvcZCxcQZCglQshts2EZCZBccbvwUddxKlje4ZCz3w8qkLWICYdS8cZCAw2e8VZCs9VHEt7sg5gR6gc4o8TVV73OAABUf7zKqoXLyeREavx7Viz8ArwVQxsDfMVSJK7o5qOqiQZDZD"

def creatfbaudience(file_path,ACCESS_TOKEN,accountid,description,audience_name):
    '''
    input:
    file_path: str, full_path, basename as the audience name on FB
        make sure the columns contains the "email_address_hash", use this col only
    
    ACCESS_TOKEN: personal token #app id not needed
    accountid: account_id starting with "act_", followed by the id str
    as Nov. 20, 2019, latest verision is 'v5.0'
    '''
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v5.0')
    myAccount = AdAccount(accountid)
    fields = []
    fbname=audience_name
    params = {
      'name': fbname,
      'subtype': 'CUSTOM',
      'description': description,
      'customer_file_source': 'USER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    
    df=pd.read_csv(file_path,dtype=str,)
    listlen = df.shape[0]
    requestUrl = 'https://graph.facebook.com/v5.0/'+ fbname + '/users'
    
    chunck_size=50000
    i_counter=0
    for df_chunck in pd.read_csv(file_path,chunksize=chunck_size,dtype=str,usecols=["email_address_hash"]):
        payload = {}
        payload['schema'] = ['EMAIL']
        df_chunck=df_chunck.rename(columns={"email_address_hash":"EMAIL"})
        df_chunck=df_chunck.drop_duplicates()
        payload_data=df_chunck['EMAIL'].apply(lambda x: [x]).tolist()

        payload['data'] = payload_data
        params = {}
        params ={'payload':payload} 
        CustomAudience(fbname).create_user(
                      fields=fields,
                      params=params,
        )
        i_counter+=1
        print(i_counter,datetime.datetime.now())
  

    return 'All done: '+str(datetime.datetime.now())

In [10]:
for local_file in list_seg_to_upload:

    creatfbaudience(file_path=local_file,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id,
                    description=os.path.basename(local_file).split(".")[0],
                    audience_name=os.path.basename(local_file).split(".")[0]
                   )

1 2019-12-18 16:31:41.587232
2 2019-12-18 16:31:48.058076
3 2019-12-18 16:31:49.995577
4 2019-12-18 16:31:56.255553
5 2019-12-18 16:31:58.748088
6 2019-12-18 16:32:00.919714
7 2019-12-18 16:32:03.038840
8 2019-12-18 16:32:09.109921
9 2019-12-18 16:32:11.246811
10 2019-12-18 16:32:17.303322
11 2019-12-18 16:32:23.890381
12 2019-12-18 16:32:26.143473
13 2019-12-18 16:32:32.284972
14 2019-12-18 16:32:38.422624
15 2019-12-18 16:32:40.409923
16 2019-12-18 16:32:42.643347
17 2019-12-18 16:32:44.675823
18 2019-12-18 16:32:50.767991
19 2019-12-18 16:32:56.878000
20 2019-12-18 16:33:02.976295
21 2019-12-18 16:33:05.122175
22 2019-12-18 16:33:07.107439
23 2019-12-18 16:33:09.050373
24 2019-12-18 16:33:15.182465
25 2019-12-18 16:33:21.731344
26 2019-12-18 16:33:27.755462
27 2019-12-18 16:33:34.331100
28 2019-12-18 16:33:40.609896
29 2019-12-18 16:33:46.752064
30 2019-12-18 16:33:52.887363
31 2019-12-18 16:33:58.987946
32 2019-12-18 16:34:04.970905
33 2019-12-18 16:34:07.548951
34 2019-12-18 16:34